In [55]:
import os

import pandas as pd

In [56]:
FileFrame = pd.read_csv('FileFrame.csv')

In [57]:
FileFrame

,originPath,structuredDicomPath
0,/trinity/home/r098375/DDS/data/HeartBrain/data...,/trinity/home/r098375/DDS/data/HeartBrain/DICO...
1,/trinity/home/r098375/DDS/data/HeartBrain/data...,NaN
2,/trinity/home/r098375/DDS/data/HeartBrain/data...,/trinity/home/r098375/DDS/data/HeartBrain/DICO...
3,/trinity/home/r098375/DDS/data/HeartBrain/data...,NaN
4,/trinity/home/r098375/DDS/data/HeartBrain/data...,NaN
...,...,...
56515,/trinity/home/r098375/DDS/data/HeartBrain/data...,/trinity/home/r098375/DDS/data/HeartBrain/DICO...
56516,/trinity/home/r098375/DDS/data/HeartBrain/data...,/trinity/home/r098375/DDS/data/HeartBrain/DICO...
56517,/trinity/home/r098375/DDS/data/HeartBrain/data...,/trinity/home/r098375/DDS/data/HeartBrain/DICO...
56518,/trinity/home/r098375/DDS/data/HeartBrain/data...,/trinity/home/r098375/DDS/data/HeartBrain/DICO...


In [58]:
def FindBrainScan(path):
    return 'MRI_SCHEDEL' in path or 'MRI_HERSENEN' in path

In [59]:
FileFrame['brainscan'] = FileFrame['originPath'].apply(FindBrainScan)

In [60]:
BrainFrame = FileFrame[FileFrame['brainscan']].copy()

In [61]:
def deriveNiftiName(path):
    path = str(path).replace('DICOM_STRUCTURED','NIFTI')
    path = path+'.nii.gz'
    if 'HeartBrain' in path:
        path = path.split('HeartBrain/')[1]
    return path

In [62]:
BrainFrame['infered_NIFTI_path'] = BrainFrame['structuredDicomPath'].apply(deriveNiftiName)

In [63]:
BrainFrame['name'] = BrainFrame['originPath'].apply(lambda x: str(x).split('/')[10])

In [64]:
BrainFrame['name'].unique()

array(['401-PCA_survey', '902-perfusion_map', '601-T1', ...,
       '701-s_B1MAP_whole_brain_1min___MTX_SENSE', '301-B0MAP_1msec',
       '1101-TRUST_ASL'], dtype=object)

In [65]:
BrainFrame['str_name'] = BrainFrame['name'].apply(lambda x:str(x).split('-',2)[1])

In [66]:
BrainFrame['str_name'].unique()

array(['PCA_survey', 'perfusion_map', 'T1', 'M0', 'asl_source',
       'mpr_flair', 'MPR_hippo_cor', 'SWI', 'QF_min_wfs', 'FLAIR',
       'SURVEY_MST', 'unknown', 'MR1', 'OT1', 'Rigid___Medium',
       'sagmip_4__2_T1w_3D_TFE_pulsate', 'VISUAL_SCOUT_SENSE',
       'cormip_4__2_T1w_3D_TFE_pulsate', 'tramip_4__2_T1w_3D_TFE_pulsate',
       'MPR_SmartBrain', 'duyn_visual_3min_SENSE',
       's_T1w_3D_TFE_pulsate_SENSE', 'VISUAL_shrtStim_SENSE',
       'MidSlabSag_T1w_3D_TFE_pulsate', 'SmartBrain', 'B1_calibration',
       'OT6', 'OT3', 'OT5', 'QF_min_wfs_120048_663', 'OT2', 'OT4',
       'WIP_QF_min_wfs_CLEAR_120458_987',
       'V3D_Brain_View_FLAIR_SHC_____sag', 'Vs_T1W_3D_TFE', 'FLAIR_extra',
       'WIP_QF_min_wfs_CLEAR_145730_619', 'SWI_extra', 's_bFFE_PVS',
       'T1_extra', 'OT7', 'VWIP_s_T1W_3D_TFE_SENSE', 'ASL_SENSE',
       'sPerfusion', 'QF_min_wfs_121740_436', 'TRUST_ASL',
       'WIP_QF_min_wfs_CLEAR_153810_368',
       'WIP_QF_min_wfs_CLEAR_151125_591',
       'duyn_halfbra

In [67]:
def rename(origName):
    on = origName.lower()
    if 'local' in on:
        return 'localizer'
    if 'qflow' in on or 'qf' in on:
        return 'qflow'
    if 'gd' in on:
        if 't1' in on or 'fspgr' in on:
            return 'T1c'
    if 'flair' in on:
        return 'flair'
    if 'dti' in on:
        return 'dti'
    if 'dwi' in on:
        return 'dwi'
    if 'adc' in on:
        return 'adc'
    if 'pwi' in on or 'perf' in on:
        return 'pwi'
    if 'pd' in on:
        return 'pd'
    if 'survey' in on:
        return 'survey'
    if 'asl' in on:
        return 'asl'
    if 'swi' in on:
        return 'swi'
    if 'hippo' in on:
        return 'hippo'
    if 't1' in on or 'fspgr' in on or 'mpr' in on:
        return 'T1w'
    if 't2' in on and not 'ot2' in on:
        return 'T2w'
    return 'other'

In [68]:
BrainFrame['renamed'] = BrainFrame['name'].apply(rename)

In [69]:
BrainFrame['renamed'].unique()

array(['survey', 'pwi', 'T1w', 'other', 'asl', 'flair', 'hippo', 'swi',
       'qflow'], dtype=object)

In [70]:
BrainFrame['renamed'].value_counts()

renamed
other     7365
survey    6112
T1w       4929
flair     3034
asl       2862
swi       2628
qflow     2592
pwi       1624
hippo      306
Name: count, dtype: int64

In [76]:
print(pd.Series(BrainFrame[BrainFrame['renamed']=='asl']['name'].unique()).to_string())

0               901-asl_source
1               702-asl_source
2               801-asl_source
3     0-OT5-SOURCE___ASL_SENSE
4              0-OT1-ASL_SENSE
5               802-asl_source
6                901-ASL_SENSE
7              1401-asl_source
8                901-TRUST_ASL
9               1001-TRUST_ASL
10             2101-asl_source
11              902-asl_source
12             1101-asl_source
13              602-asl_source
14             1001-asl_source
15             0-OT3-ASL_SENSE
16             1002-asl_source
17             0-OT4-ASL_SENSE
18             0-OT5-ASL_SENSE
19                     801-ASL
20    0-OT3-SOURCE___ASL_SENSE
21             0-OT2-ASL_SENSE
22              701-asl_source
23             1501-asl_source
24               801-ASL_extra
25                 1201-ASL_IR
26        1101-ASL_PLD700_8phs
27               701-ASL_SENSE
28                     701-ASL
29             2401-asl_source
30                 1101-ASL_IR
31        1001-ASL_PLD700_8phs
32      

In [79]:
BrainFrame['renamed'].unique()

array(['survey', 'pwi', 'T1w', 'other', 'asl', 'flair', 'hippo', 'swi',
       'qflow'], dtype=object)

In [84]:
def CreateIndivudalToUnifiedNameDocument(df, outfile):
    with open(outfile, 'w') as f:
        names = df['renamed'].unique()
        for name in names:
            nameMatching = df[df['renamed']==name]['name'].unique()
            nameMatching = pd.Series(nameMatching).to_string()
            f.write(f'{name}:\n{nameMatching}\n\n{"-"*25}\n')

In [85]:
outfile = 'NameMatching.txt'
CreateIndivudalToUnifiedNameDocument(BrainFrame, outfile)

In [ ]:
labeldict = {"T1":0,"T1_c":1,"T2":2,"T2-FLAIR":3,"FS":4,"PD":5,
             "SWI":6, "GRE":7, "T2*":8, "DWI":9, "ADC":10, "BOLD":11,
             "FlowSensitve":12,"PWI":13,"ASL":14,"Hippo":15,
             "DTI":16,"Other":17}
def assignLabel(dataframe):
    description = dataframe['description']
    try:
        weight = dataframe['weighting']
    except:
        weight = None
    otherCategories = ['fieldmap','scout','calibration','phase','localizer','nan','NotMri','pet','CT','e2','minIP','moco','mpr','other','survey', 'qflow'] #moco is a motion corrected series
    if description in otherCategories:
        return 17
    if description =='T1w':
        return 0
    if description =='T1c':
        return 1
    if weight:
        if description =='T2w':
            if weight=='T2':
                return 2
            if weight=='PD':
                return 5
    if description == 'T2w':
        return 2
    if 'flair' in description.lower():
        return 3
    if description =='FS':
        return 4
    if description =='pd':
        return 5
    if description =='swi':
        return 6
    if description.lower() =='gre':
        return 7
    if description =='T2star':
        return 8
    if description =='dwi':
        return 9
    if description =='adc':
        return 10
    if description =='bold':
        return 11
    if description =='angio': #flowsensitive
        return 12
    if description =='pwi' or description=='cbf':
        return 13
    if 'asl' in description:
        return 14
    if description == 'hippo':
        return 15
    if description == 'dti':
        return 16
    return -1

In [ ]:
BrainFrame['description'] = BrainFrame['renamed']

In [ ]:
BrainFrame['numLabel'] = BrainFrame.apply(assignLabel,axis=1)

In [ ]:
BrainFrame['numLabel'].value_counts()

In [ ]:
LabelFrame = pd.read_csv('Labels.txt',names=['ID','label','extra'],sep='\t', dtype={'ID':str,'label':int,'extra':int})

In [ ]:
def extractNiftiName(name):
    name = name.rsplit('__',1)[0]
    name = name.replace('NIFTI_SLICES','NIFTI')
    name = name.split('_Scan')[0]
    name = f'{name}.nii.gz'
    if len(str(name).split('__')) > 3:
        return None
    return name

In [ ]:
LabelFrame['nifti_name'] = LabelFrame['ID'].apply(extractNiftiName)

In [ ]:
len(LabelFrame)

In [ ]:
LabelFrame.head()

In [ ]:
BrainFrame.head()

In [ ]:
temp = LabelFrame.merge(BrainFrame, how='inner', left_on='nifti_name', right_on="infered_NIFTI_path")

In [ ]:
temp

In [ ]:
temp['description'].value_counts()

In [ ]:
temp['label'] = temp['numLabel']

In [ ]:
FinalLabelFrame = temp[['ID','label','extra']].copy()

In [ ]:
FinalLabelFrame

In [ ]:
appendstring = '/trinity/home/r098375/DDS/data/HeartBrain/'
def fixPath(ID):
    if not 'r098375' in ID:
        return appendstring+ID
    return ID

In [ ]:
FinalLabelFrame['ID'] = FinalLabelFrame.ID.apply(fixPath)

In [ ]:
def extractSubject(ID):
    return ID.split('/')[8]

In [ ]:
FinalLabelFrame['subject'] = FinalLabelFrame['ID'].apply(extractSubject)

In [ ]:
FinalLabelFrame['subject'].unique()

In [ ]:
subjects = FinalLabelFrame['subject'].unique()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(subjects, test_size=0.1, shuffle=True, random_state=42)

In [ ]:
def inList(subject, list):
    return subject in list

In [ ]:
FinalLabelFrame['trainsplit'] = FinalLabelFrame['subject'].apply(inList, list=train)

In [ ]:
trainLabelFrame = FinalLabelFrame[FinalLabelFrame['trainsplit']][['ID','label','extra']]

In [ ]:
testLabelFrame = FinalLabelFrame[FinalLabelFrame['trainsplit']==False][['ID','label','extra']]

In [ ]:
trainLabelFrame['label'].value_counts()

In [ ]:
testLabelFrame['label'].value_counts()

In [ ]:
FinalLabelFrame.to_csv('HeartBrain_Labels_extensive2.txt',sep='\t', index=False, header=False)
testLabelFrame.to_csv('HeartBrain_testLabels_extensive2.txt',sep='\t', index=False, header=False)
trainLabelFrame.to_csv('HeartBrain_trainLabels_extensive2.txt',sep='\t', index=False, header=False)